In [124]:
import numpy as np
#import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
import pydot
from sklearn import tree
from sklearn import cross_validation
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from sklearn.externals.six import StringIO  

In [125]:
#
# GOAL: fit a decision tree to predict one of 3 iris species 
# based on sepal and petal length and width and explore, if 
# needed, reduced error pruning vs post-pruning.
#
# • Expand tree using a 90%/10% training/test set scheme by
#   varying the maximum depth. Reduce overfitting by con-
#   straining minimum samples_split to 5% of sample to be 
#   fit (90%). Capture accuracy of training/test sets and 
#   complexity according to maximum number of nodes. 
#   Variables and arrays associated with this simple method
#   will have suffix _expd
# • Tune using K-fold cross-validation (CV) on 80%/10%/10%
#   training/CV/test data. Tuning trees on the 10% holdout
#   effectively constitutes pruning mechanism. Capture 
#   tuning and test set accuracy scores. Variables and 
#   arrays associated with the tuning will have suffix 
#   _kfld
# • _tr refers to tr sets and _te refers to test sets
# • For understandability, two separate loops generate 
#   the two different methods. Results are plotted on
#   a single graph

In [126]:
# Preliminary exploratory data analysis
#
# Load Fisher's iris data and verify dimensions
data_set_name = "Fisher's Iris Data"
iris = load_iris()
samples = iris.data.shape[0]
features = iris.data.shape[1]
classes = iris.target_names.shape[0]
print samples

150


In [127]:
#df = pd.read_csv("iris.csv") #Reading the dataset in a dataframe using Pandas
#df.head(10)

In [128]:
# Separate data into randomly selected training/validation and test
# sets keeping the test set for final model testing. Since the iris
# data is so small will do a K-fold cv but will still do final
# test for some percent reserved for hold_out.
#
samples_te =.9
rand = 21
X_tr_expd, X_te_expd, y_tr_expd, y_te_expd = cross_validation.train_test_split(
    iris.data, iris.target, test_size=samples_te, random_state=rand)
print(X_tr_expd.shape[0])

15


In [129]:
# Initialize values for the maximum depth and counters for complexity
# and accuracy
#
tot_nodes_expd = [] # attrib # nodes for DT by depth; may differ for CV
tot_nodes_kfld = [] # attrib # nodes for DT by depth; may differ for CV (**FIX**)
acc_tr_expd = []  # accuracy from simple DT build on training set
acc_te_expd = []  # accuracy from simple DT build on test set
acc_tr_kfld = []  # accuracy from kfolds on training set
acc_te_kfld = []  # accuracy from kfolds on hold out set

In [113]:
#folds = 5  # choose your folds
#kfold = cross_validation.KFold(X_train.shape[0],n_folds=folds)

In [130]:
max_depth = samples/(features*classes)  # heuristic to honesty build out tree
max_depths = range(1,max_depth+1) # fix indices to correspond with nodes in loop
print max_depth

12


In [115]:
max_depths = range(1,max_depth+1) # fix indices to correspond with nodes
max_depth_obs = 1
#
for depth in max_depths:
    clf = tree.DecisionTreeClassifier(max_depth=depth)
    kf_clf = clf # for use in inner loop for k-fold CV
    clf = clf.fit(X_tr_expd, y_tr_expd) # fully grown tree s.t. min samples
# Pull attribute for test/train model and calculate errors
    max_depth_obs = max(max_depth_obs, clf.tree_.max_depth)
    tot_nodes_expd.append(clf.tree_.node_count) # works for test set too
    pred_y_tr_expd = clf.predict(X_tr_expd)
    acc_tr_expd.append(accuracy_score(y_tr_expd, pred_y_tr_expd))
    pred_y_te_expd = clf.predict(X_te_expd)
    acc_te_expd.append(accuracy_score(y_te_expd, pred_y_te_expd))

print("FINAL TRAIN-TEST TREE: max_depth=", max_depth_obs)
print("tot_nodes=", tot_nodes_expd[0:max_depth])
print("acc_train=", np.around(acc_tr_expd[0:max_depth],4))
print("Train Confusion Matrix ", confusion_matrix(y_tr_expd, pred_y_tr_expd))
print("acc_test=", np.around(acc_te_expd[0:max_depth],4))
print("Test Confusion Matrix", confusion_matrix(y_te_expd,pred_y_te_expd))
#print("acc_kfold_tr=", np.around(acc_kfold_tr[0:depth],4))
#print("acc_kfold_te=", np.around(acc_kfold_te[0:depth],4))

('FINAL TRAIN-TEST TREE: max_depth=', 2)
('tot_nodes=', [3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])
('acc_train=', array([ 0.8667,  1.    ,  1.    ,  1.    ,  1.    ,  1.    ,  1.    ,
        1.    ,  1.    ,  1.    ,  1.    ,  1.    ]))
('Train Confusion Matrix ', array([[2, 0, 0],
       [0, 6, 0],
       [0, 0, 7]]))
('acc_test=', array([ 0.5852,  0.7259,  0.9407,  0.7259,  0.9407,  0.7259,  0.9407,
        0.9407,  0.9407,  0.7259,  0.9407,  0.9407]))
('Test Confusion Matrix', array([[48,  0,  0],
       [ 0, 42,  2],
       [ 0,  6, 37]]))


In [116]:
# Init for fold loop
#    acc_k_tr = [] # init to track kfold training errors for each depth
#    acc_k_te = []  # init to track kfold testing errors for each depth
#    best_kf_clf = np.zeros[[1,maxdepths]]
# train K-fold and get kfold and test accuracy; get best fold tree 
#    for train, test in kfold:
#        X_ktrain, X_ktest = X_train[train], X_train[test] 
#        y_ktrain, y_ktest = y_train[train], y_train[test]       
#        kf_clf.fit(X_ktrain,y_ktrain)
#        pred_y_ktest = clf.predict(X_ktest) 
#        acc_k_tr.append(accuracy_score(y_ktest,pred_y_ktest)) # acc of kfold training
#        print(acc_k_tr)
#        pred_y_test_final = clf.predict(X_test) 
#        acc_k_te.append(accuracy_score(y_test,pred_y_test_meta))
#    acc_kfold_tr.append(np.mean(acc_k_tr))
#    acc_kfold_te.append(np.mean(acc_k_te))

In [117]:
dot_data = StringIO() 
tree.export_graphviz(clf, out_file=dot_data, feature_names=iris.feature_names)
dot_data.getvalue()

'digraph Tree {\n0 [label="petal length (cm) <= 4.9500\\ngini = 0.604444444444\\nsamples = 15", shape="box"] ;\n1 [label="petal width (cm) <= 0.6500\\ngini = 0.375\\nsamples = 8", shape="box"] ;\n0 -> 1 ;\n2 [label="gini = 0.0000\\nsamples = 2\\nvalue = [ 2.  0.  0.]", shape="box"] ;\n1 -> 2 ;\n3 [label="gini = 0.0000\\nsamples = 6\\nvalue = [ 0.  6.  0.]", shape="box"] ;\n1 -> 3 ;\n4 [label="gini = 0.0000\\nsamples = 7\\nvalue = [ 0.  0.  7.]", shape="box"] ;\n0 -> 4 ;\n}'

In [118]:
graph = pydot.graph_from_dot_data(dot_data.getvalue())

In [119]:
max_depth_str = str(max_depth)
test_size = str(samples_te)

In [120]:
rand_num = str(rand)

In [121]:
filename = data_set_name + "_test_" + test_size + "%_max_depth_"
filename = filename + max_depth_str + "_rand_" + rand_num
filename_dt = filename + "_dt"
print filename_dt

Fisher's Iris Data_test_0.9%_max_depth_12_rand_21_dt


In [122]:
graph.write_pdf("iris.pdf")

True

In [123]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(tot_nodes_expd, acc_tr_expd, label="Training Set")
ax.plot(tot_nodes_expd, acc_te_expd, label="Test Set")
#ax.plot(tot_nodes_train, acc_kfold_tr, label="Kfold Set")
#ax.plot(tot_nodes_train, acc_kfold_te, label="Pruned Test Set")
ax.set_xlabel('Size of Tree (# nodes)')
ax.set_ylabel('Accuracy')
ax.set_xlim(1,20)
ax.set_xticks(range(21))
ax.set_ylim(0.5,1.05)
ax.set_title(filename)
filename_png = filename + ".png"
ax.yaxis.grid(color='gray', linestyle='dashed')
ax.xaxis.grid(color='gray', linestyle='dashed')
ax.legend(loc='lower right', fontsize=12)
#plt.show()
plt.savefig(filename_png)